In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/book-price-machinehack/Data_Train.xlsx
/kaggle/input/book-price-machinehack/Data_Test.xlsx


Exploration des données

In [2]:
df= pd.read_excel('../input/book-price-machinehack/Data_Train.xlsx')
df_test= pd.read_excel('../input/book-price-machinehack/Data_Test.xlsx')
df= df.merge(df_test)                     
df.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,Peace of Mind,Thich Nhat Hanh,"Paperback,– 5 Feb 2015",5.0 out of 5 stars,1 customer review,"In Peace of Mind, Zen Master Thich Nhat Hanh r...",Action & Adventure (Books),Action & Adventure,342.00
1,The Scarecrows' Wedding,Julia Donaldson,"Paperback,– Import, 7 Jul 2016",3.6 out of 5 stars,4 customer reviews,The Scarecrows' Wedding is a wonderfully h...,Action & Adventure (Books),Action & Adventure,334.00
2,Prey,Michael Crichton,"Paperback,– Import, 4 Aug 2003",4.5 out of 5 stars,16 customer reviews,From the Number One international bestselling ...,Action & Adventure (Books),Action & Adventure,385.00
3,"Innovators: How a Group of Inventors, Hackers,...",Walter Isaacson,"Paperback,– Import, 6 Oct 2015",4.6 out of 5 stars,74 customer reviews,Following his blockbuster biography of Steve J...,Action & Adventure (Books),Action & Adventure,407.00
4,The Power of Love: What Does It Take for Love ...,Osho,"Paperback,– Import, 19 Jul 2016",4.7 out of 5 stars,4 customer reviews,Explore the emotional sensations of the many f...,Spirituality,Action & Adventure,253.69


In [3]:
df.shape

(36, 9)

In [4]:
df.describe()

,Price
count,36.000000
mean,370.560278
std,348.456410
min,118.000000
25%,245.517500
50%,280.500000
75%,373.000000
max,2263.000000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 0 to 35
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Title         36 non-null     object 
 1   Author        36 non-null     object 
 2   Edition       36 non-null     object 
 3   Reviews       36 non-null     object 
 4   Ratings       36 non-null     object 
 5   Synopsis      36 non-null     object 
 6   Genre         36 non-null     object 
 7   BookCategory  36 non-null     object 
 8   Price         36 non-null     float64
dtypes: float64(1), object(8)
memory usage: 2.8+ KB


In [6]:
df['Reviews'] = df.Reviews.apply(lambda r: float(r.split()[0]))
df['Ratings']= df.Ratings.str.extract('(\d+)')
df["Ratings"] = df.Ratings.astype(float)

In [7]:
df.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,Peace of Mind,Thich Nhat Hanh,"Paperback,– 5 Feb 2015",5.0,1.0,"In Peace of Mind, Zen Master Thich Nhat Hanh r...",Action & Adventure (Books),Action & Adventure,342.00
1,The Scarecrows' Wedding,Julia Donaldson,"Paperback,– Import, 7 Jul 2016",3.6,4.0,The Scarecrows' Wedding is a wonderfully h...,Action & Adventure (Books),Action & Adventure,334.00
2,Prey,Michael Crichton,"Paperback,– Import, 4 Aug 2003",4.5,16.0,From the Number One international bestselling ...,Action & Adventure (Books),Action & Adventure,385.00
3,"Innovators: How a Group of Inventors, Hackers,...",Walter Isaacson,"Paperback,– Import, 6 Oct 2015",4.6,74.0,Following his blockbuster biography of Steve J...,Action & Adventure (Books),Action & Adventure,407.00
4,The Power of Love: What Does It Take for Love ...,Osho,"Paperback,– Import, 19 Jul 2016",4.7,4.0,Explore the emotional sensations of the many f...,Spirituality,Action & Adventure,253.69


Identification des variables catégoriques, ainsiu que leur importance dans le datasat:

In [8]:
# Get list of categorical variables
s = (df.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)

Categorical variables:
['Title', 'Author', 'Edition', 'Synopsis', 'Genre', 'BookCategory']


In [9]:
# Get number of unique entries in each column with categorical data
object_nunique = list(map(lambda col: df[col].nunique(), object_cols))
d = dict(zip(object_cols, object_nunique))

# Print number of unique entries by column, in ascending order
sorted(d.items(), key=lambda x: x[1])

[('BookCategory', 1),
 ('Genre', 5),
 ('Author', 34),
 ('Edition', 34),
 ('Title', 36),
 ('Synopsis', 36)]

Encodage one-hot

In [10]:
df_1= pd.get_dummies(df, columns= ['Genre', 'BookCategory'])
df_1.head()

,Title,Author,Edition,Reviews,Ratings,Synopsis,Price,Genre_Action & Adventure (Books),Genre_Encyclopaedias & Reference Works (Books),Genre_Neuroscience,Genre_Short Stories (Books),Genre_Spirituality,BookCategory_Action & Adventure
0,Peace of Mind,Thich Nhat Hanh,"Paperback,– 5 Feb 2015",5.0,1.0,"In Peace of Mind, Zen Master Thich Nhat Hanh r...",342.00,1,0,0,0,0,1
1,The Scarecrows' Wedding,Julia Donaldson,"Paperback,– Import, 7 Jul 2016",3.6,4.0,The Scarecrows' Wedding is a wonderfully h...,334.00,1,0,0,0,0,1
2,Prey,Michael Crichton,"Paperback,– Import, 4 Aug 2003",4.5,16.0,From the Number One international bestselling ...,385.00,1,0,0,0,0,1
3,"Innovators: How a Group of Inventors, Hackers,...",Walter Isaacson,"Paperback,– Import, 6 Oct 2015",4.6,74.0,Following his blockbuster biography of Steve J...,407.00,1,0,0,0,0,1
4,The Power of Love: What Does It Take for Love ...,Osho,"Paperback,– Import, 19 Jul 2016",4.7,4.0,Explore the emotional sensations of the many f...,253.69,0,0,0,0,1,1


In [11]:
df_2= df_1.drop(['Title', 'Author','Synopsis','Edition'],1, inplace= False)
df_2.head()

,Reviews,Ratings,Price,Genre_Action & Adventure (Books),Genre_Encyclopaedias & Reference Works (Books),Genre_Neuroscience,Genre_Short Stories (Books),Genre_Spirituality,BookCategory_Action & Adventure
0,5.0,1.0,342.00,1,0,0,0,0,1
1,3.6,4.0,334.00,1,0,0,0,0,1
2,4.5,16.0,385.00,1,0,0,0,0,1
3,4.6,74.0,407.00,1,0,0,0,0,1
4,4.7,4.0,253.69,0,0,0,0,1,1


In [12]:
df_2.describe()

,Reviews,Ratings,Price,Genre_Action & Adventure (Books),Genre_Encyclopaedias & Reference Works (Books),Genre_Neuroscience,Genre_Short Stories (Books),Genre_Spirituality,BookCategory_Action & Adventure
count,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.000000,36.0
mean,4.275000,20.833333,370.560278,0.888889,0.027778,0.027778,0.027778,0.027778,1.0
std,0.510112,29.821852,348.456410,0.318728,0.166667,0.166667,0.166667,0.166667,0.0
min,2.700000,1.000000,118.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0
25%,3.975000,4.000000,245.517500,1.000000,0.000000,0.000000,0.000000,0.000000,1.0
50%,4.400000,13.000000,280.500000,1.000000,0.000000,0.000000,0.000000,0.000000,1.0
75%,4.700000,17.500000,373.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.0
max,5.000000,143.000000,2263.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0


Déploiement d'un modèle avec Pycaret

In [13]:
!pip install pycaret

     |████████████████████████████████| 252 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 258 kB 9.5 MB/s eta 0:00:01
     |████████████████████████████████| 13.9 MB 13.9 MB/s eta 0:00:01
     |████████████████████████████████| 96 kB 3.1 MB/s  eta 0:00:01
     |████████████████████████████████| 64 kB 2.1 MB/s  eta 0:00:01
     |████████████████████████████████| 3.1 MB 56.8 MB/s eta 0:00:01


     |████████████████████████████████| 1.1 MB 53.5 MB/s eta 0:00:01
     |████████████████████████████████| 6.0 MB 46.1 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 5.7 MB/s  eta 0:00:01
     |████████████████████████████████| 326 kB 55.8 MB/s eta 0:00:01
     |████████████████████████████████| 49 kB 3.9 MB/s  eta 0:00:01
     |████████████████████████████████| 2.1 MB 44.0 MB/s eta 0:00:01


     |████████████████████████████████| 121 kB 57.0 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 2.3 MB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 2.4 MB/s  eta 0:00:01
  Created wheel for pyod: filename=pyod-0.8.3-py3-none-any.whl size=110347 sha256=6ce29a7b0c070a2ed70276c4af703ec066a2842ae69d33b4d5ba98db97c72112
  Stored in directory: /root/.cache/pip/wheels/fc/fc/77/6e530134c9ee2b45ef0840f0c8046b3be595624881cf533d7a
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158154 sha256=f9ec3d7d97ebee9cb7fa2f497d9bc11a25143148c014a6ae66ab506e45612e00
  Stored in directory: /root/.cache/pip/wheels/be/5d/0a/9e13f53f4f5dfb67cd8d245bb7cdffe12f135846f491a283e3
  Created wheel for querystring-parser: filename=querystring_parser-1.2.4-py3-none-any.whl size=7076 sha256=5dcf842190236cf6de5393bd238374360094dd4b0f9a435e539ef6b379f35672
  Stored in directory: /root/.cache/pip/wheels/69/38/7a/072b5863ca334d012821a287fd1d066cea33abdcda3ef2f8

  Attempting uninstall: pandas-profiling
    Found existing installation: pandas-profiling 2.6.0
    Uninstalling pandas-profiling-2.6.0:
      Successfully uninstalled pandas-profiling-2.6.0
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 1.3.16
    Uninstalling SQLAlchemy-1.3.16:
      Successfully uninstalled SQLAlchemy-1.3.16
  Attempting uninstall: alembic
    Found existing installation: alembic 1.4.2
    Uninstalling alembic-1.4.2:
      Successfully uninstalled alembic-1.4.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

pandas-profiling 2.9.0 requires seaborn>=0.10.1, but you'll have seaborn 0.10.0 which is incompatible.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --up

In [14]:
from pycaret.regression import *
PTA= setup(df_2, 
           target= "Price")

 
Setup Succesfully Completed.


,Description,Value
0,session_id,662
1,Transform Target,False
2,Transform Target Method,None
3,Original Data,"(36, 9)"
4,Missing Values,False
5,Numeric Features,8
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [15]:
compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
0,Support Vector Machine,156.5868,155180.2428,219.6214,-2.3454,0.4385,0.3035,0.0034
1,Decision Tree,204.2643,185742.1068,270.2507,-6.9218,0.5813,0.5561,0.0049
2,Huber Regressor,182.2206,154451.4832,239.5512,-7.2676,0.5601,0.3963,0.0428
3,Light Gradient Boosting Machine,193.0319,155165.3293,249.2934,-11.8455,0.5291,0.5324,0.0081
4,Bayesian Ridge,193.0334,155165.3366,249.2943,-11.8458,0.5291,0.5324,0.0079
5,Elastic Net,200.9858,154648.2162,257.6936,-15.1342,0.5491,0.5539,0.0048
6,TheilSen Regressor,210.3058,162826.7153,272.6090,-16.4767,0.6826,0.5057,0.7522
7,Orthogonal Matching Pursuit,202.0584,158829.8301,267.2369,-17.3626,0.5799,0.5407,0.0040
8,Random Forest,191.3793,165255.3361,250.3443,-20.4483,0.5256,0.4841,0.3085
9,Ridge Regression,211.4419,157024.9562,274.0631,-20.9665,0.6154,0.5787,0.0056


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [16]:
SVM= create_model('svm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,37.1686,2372.7870,48.7113,-1.4370,0.2019,0.1738
1,56.2054,4079.4541,63.8706,-0.0123,0.2159,0.1930
2,685.8965,1320000.9145,1148.9129,-0.4505,1.2267,0.4006
3,137.5059,20589.3974,143.4901,-0.0660,0.5795,0.6425
4,60.0416,5890.6670,76.7507,-1.5804,0.3483,0.3413
5,43.0178,3204.2957,56.6065,-1.4052,0.2390,0.2130
6,316.2381,168728.9393,410.7663,-0.6873,0.8200,0.4645
7,135.0608,20358.8155,142.6843,-8.6214,0.4127,0.3240
8,25.6116,811.7221,28.4907,-4.1950,0.0985,0.0848
9,69.1222,5765.4356,75.9305,-4.9994,0.2425,0.1976


In [17]:
tuned_lr = tune_model(SVM)

NameError: name 'LR' is not defined

In [ ]:
# plot a model 
plot_model(SVM)

In [ ]:
# predictions on hold-out set
SVM_pred_holdout = predict_model(SVM)


In [ ]:
# evaluate a model 
evaluate_model(SVM)

In [ ]:
# finalize model
lr_final = finalize_model(SVM)

In [ ]:
# save a model
save_model(SVM,'SVM_model_21092020')